In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
train_data_1 = pd.read_csv('train_preprocess_v1.csv').values
test_data_1 = pd.read_csv('test_preprocess_v1.csv').values

In [3]:
train_data_2 = pd.read_csv('train_preprocess_v2.csv').values
test_data_2 = pd.read_csv('test_preprocess_v2.csv').values

In [4]:
train_data_3 = pd.read_csv('train_preprocess_v3.csv').values
test_data_3 = pd.read_csv('test_preprocess_v3.csv').values

In [5]:
train_data_4 = pd.read_csv('train_preprocess_v4.csv').values
test_data_4 = pd.read_csv('test_preprocess_v4.csv').values

In [6]:
train_data_1.shape, train_data_2.shape, train_data_3.shape, train_data_4.shape

((7000, 920), (7000, 1537), (7000, 38), (7000, 1752))

In [7]:
X_train = np.concatenate([train_data_1, train_data_2, train_data_3, train_data_4], axis=1)
X_test = np.concatenate([test_data_1, test_data_2, test_data_3, test_data_4], axis=1)

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

label_df = pd.read_csv('label.csv', header=None)
y_train = label_df[0].values

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [9]:
from feature_selector import FeatureSelector

fs = FeatureSelector(data=pd.DataFrame(X_train), labels=y_train)

fs.identify_collinear(correlation_threshold=0.90)

train_new_df = fs.remove(methods = ['collinear'])
test_new_df = pd.DataFrame(X_test)[train_new_df.columns]

502 features with a correlation magnitude greater than 0.90.

Removed 502 features.


In [21]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

selector = VarianceThreshold(0.2)
X_train = selector.fit_transform(imputer.fit_transform(train_new_df.replace([np.inf, -np.inf], np.nan).values))
X_test = selector.transform(imputer.fit_transform(test_new_df.replace([np.inf, -np.inf], np.nan).values))

In [22]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE, VarianceThreshold

rfe = RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.9000000000000001, n_estimators=100), step=0.15000000000000002)

rfe.fit(X_train, y_train)

RFE(estimator=ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                   class_weight=None, criterion='entropy',
                                   max_depth=None,
                                   max_features=0.9000000000000001,
                                   max_leaf_nodes=None, max_samples=None,
                                   min_impurity_decrease=0.0,
                                   min_impurity_split=None, min_samples_leaf=1,
                                   min_samples_split=2,
                                   min_weight_fraction_leaf=0.0,
                                   n_estimators=100, n_jobs=None,
                                   oob_score=False, random_state=None,
                                   verbose=0, warm_start=False),
    n_features_to_select=None, step=0.15000000000000002, verbose=0)

In [26]:
X_train[:, rfe.support_].shape, X_test[:, rfe.support_].shape

((7000, 1640), (2000, 1640))

In [27]:
X_train = X_train[:, rfe.support_]
X_test = X_test[:, rfe.support_]

In [28]:
rfe = RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.9000000000000001, n_estimators=100), step=0.15000000000000002)

rfe.fit(X_train, y_train)

RFE(estimator=ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                   class_weight=None, criterion='entropy',
                                   max_depth=None,
                                   max_features=0.9000000000000001,
                                   max_leaf_nodes=None, max_samples=None,
                                   min_impurity_decrease=0.0,
                                   min_impurity_split=None, min_samples_leaf=1,
                                   min_samples_split=2,
                                   min_weight_fraction_leaf=0.0,
                                   n_estimators=100, n_jobs=None,
                                   oob_score=False, random_state=None,
                                   verbose=0, warm_start=False),
    n_features_to_select=None, step=0.15000000000000002, verbose=0)

In [29]:
X_train[:, rfe.support_].shape, X_test[:, rfe.support_].shape

((7000, 820), (2000, 820))

In [30]:
X_train = X_train[:, rfe.support_]
X_test = X_test[:, rfe.support_]

In [52]:
rfe = RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.9000000000000001, n_estimators=100), step=0.15000000000000002)

rfe.fit(X_train, y_train)

RFE(estimator=ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                   class_weight=None, criterion='entropy',
                                   max_depth=None,
                                   max_features=0.9000000000000001,
                                   max_leaf_nodes=None, max_samples=None,
                                   min_impurity_decrease=0.0,
                                   min_impurity_split=None, min_samples_leaf=1,
                                   min_samples_split=2,
                                   min_weight_fraction_leaf=0.0,
                                   n_estimators=100, n_jobs=None,
                                   oob_score=False, random_state=None,
                                   verbose=0, warm_start=False),
    n_features_to_select=None, step=0.15000000000000002, verbose=0)

In [53]:
X_train[:, rfe.support_].shape, X_test[:, rfe.support_].shape

((7000, 102), (2000, 102))

In [54]:
X_train = X_train[:, rfe.support_]
X_test = X_test[:, rfe.support_]

In [55]:
import lightgbm as lgb

In [80]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [85]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=2020, shuffle=True)

model_list_1 = []
score_list = []
for train_index, test_index in kf.split(X_train):
    model = lgb.LGBMClassifier(n_estimators=1000, objective='multiclass', num_leaves=63,
                               max_depth=7, learning_rate=0.03, subsample=0.8, colsample_bytree=0.8)
    eval_set = (X_train[test_index], y_train[test_index])
    model.fit(X=X_train[train_index], y=y_train[train_index], eval_metric=evaluate_macroF1_lgb,
              eval_set=eval_set, early_stopping_rounds=100, verbose=100)
    model_list_1.append(model)
    score_list.append(f1_score(y_train[test_index], model.predict(X_train[test_index]), average='macro'))
    
print(score_list)
print(np.mean(score_list), np.std(score_list))

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.363979	valid_0's macroF1: 0.82394
[200]	valid_0's multi_logloss: 0.322392	valid_0's macroF1: 0.838802
[300]	valid_0's multi_logloss: 0.307682	valid_0's macroF1: 0.838982
[400]	valid_0's multi_logloss: 0.301665	valid_0's macroF1: 0.84607
[500]	valid_0's multi_logloss: 0.30116	valid_0's macroF1: 0.852559
[600]	valid_0's multi_logloss: 0.303997	valid_0's macroF1: 0.85733
Early stopping, best iteration is:
[514]	valid_0's multi_logloss: 0.300795	valid_0's macroF1: 0.852559
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.339705	valid_0's macroF1: 0.844847
[200]	valid_0's multi_logloss: 0.287422	valid_0's macroF1: 0.858506
[300]	valid_0's multi_logloss: 0.272318	valid_0's macroF1: 0.867367
[400]	valid_0's multi_logloss: 0.264306	valid_0's macroF1: 0.874621
[500]	valid_0's multi_logloss: 0.259068	valid_0's macroF1: 0.876463
Early stopping, best iteratio

In [86]:
from sklearn.metrics import f1_score

def evaluate_macroF1_xgb(predictions, dtrain):
    labels = dtrain.get_label()
    pred_labels = predictions.reshape(len(np.unique(labels)), -1).argmax(axis=0)
    f1 = f1_score(labels, pred_labels, average='macro')
    return 'macroF1', 1-f1

import xgboost as xgb

kf = KFold(n_splits=5, random_state=2020, shuffle=True)

model_list_2 = []
score_list = []
for train_index, test_index in kf.split(X_train):
    model = xgb.XGBClassifier(n_estimators=1000, objective='multi:softmax', num_leaves=63,
                               max_depth=7, learning_rate=0.03, subsample=0.8, colsample_bytree=0.8)
    eval_set = [(X_train[test_index], y_train[test_index])]
    model.fit(X_train[train_index], y_train[train_index], eval_set=eval_set,
              early_stopping_rounds=100, verbose=100)
    model_list_2.append(model)
    score_list.append(f1_score(y_train[test_index], model.predict(X_train[test_index]), average='macro'))
    
print(score_list)
print(np.mean(score_list), np.std(score_list))

[0]	validation_0-merror:0.174286
Will train until validation_0-merror hasn't improved in 100 rounds.
[100]	validation_0-merror:0.117143
[200]	validation_0-merror:0.113571
[300]	validation_0-merror:0.107143
[400]	validation_0-merror:0.107143
Stopping. Best iteration:
[343]	validation_0-merror:0.105714

[0]	validation_0-merror:0.170714
Will train until validation_0-merror hasn't improved in 100 rounds.
[100]	validation_0-merror:0.119286
[200]	validation_0-merror:0.113571
[300]	validation_0-merror:0.108571
[400]	validation_0-merror:0.105714
[500]	validation_0-merror:0.107143
Stopping. Best iteration:
[400]	validation_0-merror:0.105714

[0]	validation_0-merror:0.182143
Will train until validation_0-merror hasn't improved in 100 rounds.
[100]	validation_0-merror:0.125714
[200]	validation_0-merror:0.115
[300]	validation_0-merror:0.110714
[400]	validation_0-merror:0.110714
Stopping. Best iteration:
[343]	validation_0-merror:0.107857

[0]	validation_0-merror:0.177143
Will train until validatio

In [91]:
result = []

for model in model_list_1:
    result.append(model.predict(X_train))
    
for model in model_list_2:
    result.append(model.predict(X_train))

In [98]:
prediction = pd.DataFrame(np.array(result).T).mode(axis=1).values